In [1]:
%cd "/Volumes/passport/Present/Work/SpaceLabWork/north_pole/NP_EKM0_TBS0_KP"

/Volumes/passport/Present/Work/SpaceLabWork/north_pole/NP_EKM0_TBS0_KP


<h1> Notes on file
    this contains 

In [134]:
import nctoolkit as nc
import os
import numpy as np
import xarray as xr
import proplot as plot 
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.path as mpath
import astropy.nddata
import scipy.signal
import math as mth
import glob2
from itertools import chain

In [3]:
imax=int(257)
jmax=int(257)
resy=90.0/jmax
resx=90.0/imax
b=(jmax+1)/2
a=(imax+1)/2
lat=np.zeros((257,257))
lon=np.zeros((257,257))

#THIS IS THE WORKING ONE
for i in range(imax) :
    for j in range(jmax):
        dd=np.sqrt((resx*(i-a))**2 + (resy*(j-b))**2) 
        latI=90-dd
        lat[i,j]= latI
        #print("this is "+ str(latI))
        if dd>0 :
            if j>= b:
                lonI=mth.acos((i-a)*resx/dd)*180/mth.pi
                lon[i,j]= lonI
                #print('lon is '+ str(lonI))
            else:
                lonI= 360-mth.acos((i-a)*resx/dd)*180/mth.pi
                lon[i,j]= lonI
                #print('lon is '+ str(lonI))

In [4]:
file_list = os.listdir("/Volumes/passport/Present/Work/SpaceLabWork/north_pole/NP_EKM0_TBS0_KP")
file_list_sp= os.listdir("/Volumes/passport/Present/Work/SpaceLabWork/south_pole/SP_EKM0_TBS1_KP")
#grab last 4 characters of the file name:
def last_7chars(x):
    return(x[-7:])


#not needed -AL
#file_list_2 = os.listdir('//Users//qjf838/Documents/SpaceData/testrun5')
#file_list_3 = os.listdir('//Users//qjf838/Documents/SpaceData/testrunsp')

#file_list_evap= os.listdir('//Users//qjf838/Documents/SpaceData/Evapnp')
#file_list_evap_sp= os.listdir('//Users//qjf838/Documents/SpaceData/Evapsp')

#file_list_prevap_ele=
#file_list_prevap_ele_sp=



def last_8chars(x):
    return(x[-8:])


def last_10chars(x):
    return(x[-10:])


In [27]:

i = 1
isp=1
xglb= int(360/5)
yglb=int(180/5)
#tglb=84 #I dont think I need it as it will iterate through i
lake_xglb= 0
lake_yglb= 0
lake_tglb= 0
long=np.zeros((xglb,yglb))
latg=np.zeros((xglb,yglb))
dist= np.zeros((257,257))
lakeg= np.zeros((xglb,yglb))
lakegsp= np.zeros((xglb,yglb))

In [10]:
#north pole
for filename in sorted(file_list, key = last_7chars):
    if filename.endswith(".dat"): # or filename.endswith(".png"):
        globals()["data" + str(i)]= pd.read_csv( filename, sep=' ', header=None, skipinitialspace=True, engine='python')
        a= np.array(globals()["data" + str(i)])
        j=0
        while j <= 7:
            globals()["data" + str(i)+ 'index_' + str(j)] = np.reshape(a[:,j], (257,257))
            #globals()["decidata" + str(i)+ 'index_' + str(j)]= astropy.nddata.block_reduce(globals()["data" + str(i)+ 'index_' + str(j)],block_size=(13,13))
            j+= 1
            
        globals()['lake'+ str(i)]=( eval('data' + str(i)+ 'index_1') > eval('data' +str(i)+'index_2'))*1
        #globals()['lake'+ str(i)]=(globals()["data" + str(i)+ 'index_' + str(1)]>globals()["data" + str(i)+ 'index_' + str(2)])*1
        #p=i h=j
        for p in range(xglb):
            long[p,1:yglb]=(p)*5.0

        for h in range(yglb):
            latg[1:xglb,h]=(h)*5.0-90

        for p in range(xglb):
            for h in range(yglb):
                if latg[p,h] >= 45: #less than -45 for sp
                    dist=np.sqrt((lat-latg[p,h])**2 + (lon-long[p,h])**2)
                    distmin= dist.min()
                    distequal = dist==distmin
                    #latr= lat> latg[p,h] and/& lat<latg[p+1,h] and lon 
                    lakeg[p,h]= globals()['lake'+ str(i)][distequal]
                    #decilakeg= astropy.nddata.block_reduce(lakeg,block_size=(13,13))

        
        np.savetxt('//Users//qjf838/Documents/SpaceData/testrun5/NPdata'+str(i)+'.txt', lakeg, delimiter=',')

        #fig = plt.figure()
        #ax = fig.add_subplot(111)  #cmap=lisbon is nice
        #ax.imshow(lakeg, aspect='auto')#, interpolation='nearest')
        #fig.savefig('//Users//qjf838/Documents/SpaceData/testrun/plot'+str(i)+'.png')

        i += 1
    else:
        continue

In [11]:
NP_files = glob2.glob('//Users//qjf838/Documents/SpaceData/testrun5' + "/*.txt")
allNP=[]
for txtfile in sorted(NP_files, key = last_8chars):
    if txtfile.endswith(".txt"): # or filename.endswith(".png"):
        df = pd.read_csv(txtfile, index_col=None, header=0)
        allNP.append(df)
frameNP = pd.concat(allNP, axis=0, ignore_index=True)

In [12]:
%cd "/Volumes/passport/Present/Work/SpaceLabWork/south_pole/SP_EKM0_TBS1_KP"

/Volumes/passport/Present/Work/SpaceLabWork/south_pole/SP_EKM0_TBS1_KP


In [15]:
#if this doesnt work, its becuase I renamed a variable SPdata, worked beforehand
for filename in sorted(file_list_sp, key = last_7chars):
    if filename.endswith(".dat"): # or filename.endswith(".png"):
        globals()["SPdata" + str(isp)]= pd.read_csv( filename, sep=' ', header=None, skipinitialspace=True, engine='python')
        a= np.array(globals()["SPdata" + str(isp)])
        j=0
        while j <= 7:
            globals()["SPdata" + str(isp)+ 'index_' + str(j)] = np.reshape(a[:,j], (257,257))
            #globals()["decidata" + str(i)+ 'index_' + str(j)]= astropy.nddata.block_reduce(globals()["data" + str(i)+ 'index_' + str(j)],block_size=(13,13))
            j+= 1
            
        globals()['lake'+ str(isp)]=( eval('data' + str(isp)+ 'index_1') > eval('data' +str(isp)+'index_2'))*1    
        #globals()['lake'+ str(i)]=(globals()["data" + str(i)+ 'index_' + str(1)]>globals()["data" + str(i)+ 'index_' + str(2)])*1
        #p=i h=j
        for p in range(xglb):
            long[p,1:yglb]=(p)*5.0

        for h in range(yglb):
            latg[1:xglb,h]=(h)*5.0-90

        for p in range(xglb):
            for h in range(yglb):
                if latg[p,h] <= -45: #less than -45 for sp
                    dist=np.sqrt((lat*(-1)-latg[p,h])**2 + (lon-long[p,h])**2)
                    distmin= dist.min()
                    distequal = dist==distmin
                    lakegsp[p,h]= globals()['lake'+ str(isp)][distequal]
                    
        np.savetxt('//Users//qjf838/Documents/SpaceData/testrunsp/data'+str(isp)+'SP.txt', lakegsp, delimiter=',')

        #fig = plt.figure()
        #ax = fig.add_subplot(111)  #cmap=lisbon is nice
        #ax.imshow(lakegsp, aspect='auto')#, interpolation='nearest')
        #fig.savefig('//Users//qjf838/Documents/SpaceData/testrun/plot'+str(i)+'.png')

        isp += 1
    else:
        continue

In [16]:
SP_files = glob2.glob('//Users//qjf838/Documents/SpaceData/testrunsp' + "/*.txt")
allSP=[]
for txtfile in sorted(SP_files, key = last_10chars):
    if txtfile.endswith(".txt"): # or filename.endswith(".png"):
        df = pd.read_csv(txtfile, index_col=None, header=0)
        allSP.append(df)
frameSP = pd.concat(allSP, axis=0, ignore_index=True)

In [17]:
#not actual code, dummy code to make sure I wrote the next cell of code correctly

#x= a-c >c #where the lakes were NP
#y= b-d >d  #where the lakes were SP

In [18]:
combined= frameNP[frameNP== 0] = frameSP[frameNP== False]

In [19]:
%cd "/Volumes/passport/Present/Work/SpaceLabWork/north_pole/NP_EKM0_TBS0_KP"

/Volumes/passport/Present/Work/SpaceLabWork/north_pole/NP_EKM0_TBS0_KP


In [20]:
for filename in sorted(file_list, key = last_7chars):
    if filename.endswith(".dat"): # or filename.endswith(".png"):
        globals()["data" + str(i)]= pd.read_csv( filename, sep=' ', header=None, skipinitialspace=True, engine='python')
        a= np.array(globals()["data" + str(i)])
        j=0
        while j <= 7:
            globals()["data" + str(i)+ 'index_' + str(j)] = np.reshape(a[:,j], (257,257))
            #globals()["decidata" + str(i)+ 'index_' + str(j)]= astropy.nddata.block_reduce(globals()["data" + str(i)+ 'index_' + str(j)],block_size=(13,13))
            j+= 1
            
        globals()['h1lake'+ str(i)]=( eval('data' + str(i)+ 'index_0') > eval('data' +str(i)+'index_2'))*1
        #globals()['lake'+ str(i)]=(globals()["data" + str(i)+ 'index_' + str(1)]>globals()["data" + str(i)+ 'index_' + str(2)])*1
        #p=i h=j
        for p in range(xglb):
            long[p,1:yglb]=(p)*5.0

        for h in range(yglb):
            latg[1:xglb,h]=(h)*5.0-90

        for p in range(xglb):
            for h in range(yglb):
                if latg[p,h] >= 45: #less than -45 for sp
                    dist=np.sqrt((lat-latg[p,h])**2 + (lon-long[p,h])**2)
                    distmin= dist.min()
                    distequal = dist==distmin
                    #latr= lat> latg[p,h] and/& lat<latg[p+1,h] and lon 
                    lakeg[p,h]= globals()['h1lake'+ str(i)][distequal]
                    #decilakeg= astropy.nddata.block_reduce(lakeg,block_size=(13,13))

        
        np.savetxt('//Users//qjf838/Documents/SpaceData/h1_lakes_NP/NPdata'+str(i)+'.txt', lakeg, delimiter=',')

        #fig = plt.figure()
        #ax = fig.add_subplot(111)  #cmap=lisbon is nice
        #ax.imshow(lakeg, aspect='auto')#, interpolation='nearest')
        #fig.savefig('//Users//qjf838/Documents/SpaceData/testrun/plot'+str(i)+'.png')

        i += 1
    else:
        continue

In [23]:
NP_h1 = glob2.glob('//Users//qjf838/Documents/SpaceData/h1_lakes_NP' + "/*.txt")
h1NP=[]
for txtfile in sorted(NP_h1, key = last_8chars):
    if txtfile.endswith(".txt"): # or filename.endswith(".png"):
        df = pd.read_csv(txtfile, index_col=None, header=0)
        h1NP.append(df)
h1_lakes_NP = pd.concat(h1NP, axis=0, ignore_index=True)

In [24]:
%cd "/Volumes/passport/Present/Work/SpaceLabWork/south_pole/SP_EKM0_TBS1_KP"

/Volumes/passport/Present/Work/SpaceLabWork/south_pole/SP_EKM0_TBS1_KP


In [28]:
#if this doesnt work, its becuase I renamed a variable SPdata, worked beforehand
for filename in sorted(file_list_sp, key = last_7chars):
    if filename.endswith(".dat"): # or filename.endswith(".png"):
        globals()["SPdata" + str(isp)]= pd.read_csv( filename, sep=' ', header=None, skipinitialspace=True, engine='python')
        a= np.array(globals()["SPdata" + str(isp)])
        j=0
        while j <= 7:
            globals()["SPdata" + str(isp)+ 'index_' + str(j)] = np.reshape(a[:,j], (257,257))
            #globals()["decidata" + str(i)+ 'index_' + str(j)]= astropy.nddata.block_reduce(globals()["data" + str(i)+ 'index_' + str(j)],block_size=(13,13))
            j+= 1
            
        globals()['h1lake'+ str(isp)]=( eval('data' + str(isp)+ 'index_0') > eval('data' +str(isp)+'index_2'))*1    
        #globals()['lake'+ str(i)]=(globals()["data" + str(i)+ 'index_' + str(1)]>globals()["data" + str(i)+ 'index_' + str(2)])*1
        #p=i h=j
        for p in range(xglb):
            long[p,1:yglb]=(p)*5.0

        for h in range(yglb):
            latg[1:xglb,h]=(h)*5.0-90

        for p in range(xglb):
            for h in range(yglb):
                if latg[p,h] <= -45: #less than -45 for sp
                    dist=np.sqrt((lat*(-1)-latg[p,h])**2 + (lon-long[p,h])**2)
                    distmin= dist.min()
                    distequal = dist==distmin
                    lakegsp[p,h]= globals()['h1lake'+ str(isp)][distequal]
                    
        np.savetxt('//Users//qjf838/Documents/SpaceData/h1_lakes_SP/data'+str(isp)+'SP.txt', lakegsp, delimiter=',')

        #fig = plt.figure()
        #ax = fig.add_subplot(111)  #cmap=lisbon is nice
        #ax.imshow(lakegsp, aspect='auto')#, interpolation='nearest')
        #fig.savefig('//Users//qjf838/Documents/SpaceData/testrun/plot'+str(i)+'.png')

        isp += 1
    else:
        continue

In [127]:
SP_h1 = glob2.glob('//Users//qjf838/Documents/SpaceData/h1_lakes_SP' + "/*.txt")
h1SP=[]
for txtfile in sorted(SP_h1, key = last_10chars):
    if txtfile.endswith(".txt"): # or filename.endswith(".png"):
        df = pd.read_csv(txtfile, index_col=None, header=0)
        h1SP.append(df)
h1_lakes_SP = pd.concat(h1SP, axis=0, ignore_index=True)

In [128]:
H1_combined= h1_lakes_NP[h1_lakes_NP== 0]= h1_lakes_SP[h1_lakes_NP== False]

In [149]:
h1_combined= H1_combined.dropna()
Combined= combined.dropna()

In [150]:
h1_combined.size

133056

In [151]:
Combined.size

133056

In [152]:
flattened_h1= list(chain.from_iterable(h1_combined))
flattened_combo= list(chain.from_iterable(Combined))

In [153]:
flattened_h1==flattened_combo

True

In [157]:
fh1= [j for sub in h1_combined for j in sub]
fc= [j for sub in Combined for j in sub]

In [108]:
a= [1,0,0,1]
b=[1,0,0,0]
apple= pd.array(a)
bannana= pd.array(b)

In [100]:
for i in apple and bannana:
    if i == 1 in apple and bannana:
        print('ay')
    else:
        print('oh no')

ay
oh no
oh no
oh no


In [109]:
for i in apple:
    for j in bannana:
        if i== j:
            print('ay')
        else:
            print('oh no')

ay
oh no
oh no
oh no
oh no
ay
ay
ay
oh no
ay
ay
ay
ay
oh no
oh no
oh no


In [ ]:
for i in h1_combined and Combined:
    if i == i in h1_combined and Combined:
        print('ay')
    else:
        print('oh no')

In [25]:
%cd "/Volumes/passport/Present/Work/SpaceLabWork/north_pole/NP_EKM0_TBS0_KP"

/Volumes/passport/Present/Work/SpaceLabWork/north_pole/NP_EKM0_TBS0_KP


In [31]:
i = 1
isp=1
xglb= int(360/5)
yglb=int(180/5)
#tglb=84 #I dont think I need it as it will iterate through i
lake_xglb= 0
lake_yglb= 0
lake_tglb= 0
long=np.zeros((xglb,yglb))
latg=np.zeros((xglb,yglb))
dist= np.zeros((257,257))
evapg= np.zeros((xglb,yglb))
evapgsp= np.zeros((xglb,yglb))

In [27]:
for filename in sorted(file_list, key = last_7chars):
    if filename.endswith(".dat"): # or filename.endswith(".png"):
        globals()["data" + str(i)]= pd.read_csv( filename, sep=' ', header=None, skipinitialspace=True, engine='python')
        a= np.array(globals()["data" + str(i)])
        j=0
        while j <= 7:
            globals()["data" + str(i)+ 'index_' + str(j)] = np.reshape(a[:,j], (257,257))
            #globals()["decidata" + str(i)+ 'index_' + str(j)]= astropy.nddata.block_reduce(globals()["data" + str(i)+ 'index_' + str(j)],block_size=(13,13))
            j+= 1
        globals()['lake'+ str(i)]=( eval('data' + str(i)+ 'index_0') > eval('data' +str(i)+'index_2'))*1    
        #globals()['evap'+ str(i)]=( eval('data' + str(isp)+ 'index_1') - eval('data' +str(isp)+'index_0'))    
        #globals()['lake'+ str(i)]=(globals()["data" + str(i)+ 'index_' + str(1)]>globals()["data" + str(i)+ 'index_' + str(2)])*1
        #p=i h=j
        for p in range(xglb):
            long[p,1:yglb]=(p)*5.0

        for h in range(yglb):
            latg[1:xglb,h]=(h)*5.0-90

        for p in range(xglb):
            for h in range(yglb):
                if latg[p,h] >= 45: #less than -45 for sp
                    dist=np.sqrt((lat-latg[p,h])**2 + (lon-long[p,h])**2)
                    distmin= dist.min()
                    distequal = dist==distmin
                    #latr= lat> latg[p,h] and/& lat<latg[p+1,h] and lon 
                    evapg[p,h]= globals()['evap'+ str(i)][distequal]
                    #decilakeg= astropy.nddata.block_reduce(lakeg,block_size=(13,13))

        
        #np.savetxt('//Users//qjf838/Documents/SpaceData/Evapnp/NPevap'+str(i)+'.txt', evapg, delimiter=',')

        #fig = plt.figure()
        #ax = fig.add_subplot(111)  #cmap=lisbon is nice
        #ax.imshow(lakeg, aspect='auto')#, interpolation='nearest')
        #fig.savefig('//Users//qjf838/Documents/SpaceData/testrun/plot'+str(i)+'.png')

        i += 1
    else:
        continue

In [28]:
NP_evap_files = glob2.glob('//Users//qjf838/Documents/SpaceData/Evapnp' + "/*.txt")
evapNP=[]
for txtfile in sorted(NP_evap_files, key = last_8chars):
    if txtfile.endswith(".txt"): # or filename.endswith(".png"):
        df = pd.read_csv(txtfile, index_col=None, header=0)
        evapNP.append(df)
evapNP = pd.concat(evapNP, axis=0, ignore_index=True)

In [29]:
%cd "/Volumes/passport/Present/Work/SpaceLabWork/south_pole/SP_EKM0_TBS1_KP"

/Volumes/passport/Present/Work/SpaceLabWork/south_pole/SP_EKM0_TBS1_KP


In [32]:

for filename in sorted(file_list_sp, key = last_7chars):
    if filename.endswith(".dat"): # or filename.endswith(".png"):
        globals()["data" + str(isp)]= pd.read_csv( filename, sep=' ', header=None, skipinitialspace=True, engine='python')
        a= np.array(globals()["data" + str(isp)])
        j=0
        while j <= 7:
            globals()["SPdata" + str(isp)+ 'index_' + str(j)] = np.reshape(a[:,j], (257,257))
            #globals()["decidata" + str(i)+ 'index_' + str(j)]= astropy.nddata.block_reduce(globals()["data" + str(i)+ 'index_' + str(j)],block_size=(13,13))
            j+= 1
            
        globals()['SPevap'+ str(i)]=( eval('SPdata' + str(isp)+ 'index_1') - eval('SPdata' +str(isp)+'index_0'))    
        #globals()['lake'+ str(i)]=(globals()["data" + str(i)+ 'index_' + str(1)]>globals()["data" + str(i)+ 'index_' + str(2)])*1
        #p=i h=j
        for p in range(xglb):
            long[p,1:yglb]=(p)*5.0

        for h in range(yglb):
            latg[1:xglb,h]=(h)*5.0-90

        for p in range(xglb):
            for h in range(yglb):
                if latg[p,h] >= 45: #less than -45 for sp
                    dist=np.sqrt((lat-latg[p,h])**2 + (lon-long[p,h])**2)
                    distmin= dist.min()
                    distequal = dist==distmin
                    #latr= lat> latg[p,h] and/& lat<latg[p+1,h] and lon 
                    evapgsp[p,h]= globals()['evap'+ str(isp)][distequal]
                    #decilakeg= astropy.nddata.block_reduce(lakeg,block_size=(13,13))


        
        #np.savetxt('//Users//qjf838/Documents/SpaceData/Evapsp/SPevap'+str(isp)+'.txt', evapg, delimiter=',')

        #fig = plt.figure()
        #ax = fig.add_subplot(111)  #cmap=lisbon is nice
        #ax.imshow(lakeg, aspect='auto')#, interpolation='nearest')
        #fig.savefig('//Users//qjf838/Documents/SpaceData/testrun/plot'+str(i)+'.png')

        isp += 1
    else:
        continue

In [33]:
SP_evap_files = glob2.glob('//Users//qjf838/Documents/SpaceData/Evapsp' + "/*.txt")
evapSP=[]
for txtfile in sorted(SP_evap_files, key = last_8chars):
    if txtfile.endswith(".txt"): # or filename.endswith(".png"):
        df = pd.read_csv(txtfile, index_col=None, header=0)
        evapSP.append(df)
evapSP = pd.concat(evapSP, axis=0, ignore_index=True)

In [34]:
evap_combined= evapNP[evapNP== 0]= evapSP[evapNP== False]

In [36]:
%cd "/Volumes/passport/Present/Work/SpaceLabWork/north_pole/NP_EKM0_TBS0_KP"

/Volumes/passport/Present/Work/SpaceLabWork/north_pole/NP_EKM0_TBS0_KP


In [37]:
i = 1
isp=1
xglb= int(360/5)
yglb=int(180/5)
#tglb=84 #I dont think I need it as it will iterate through i
lake_xglb= 0
lake_yglb= 0
lake_tglb= 0
long=np.zeros((xglb,yglb))
latg=np.zeros((xglb,yglb))
dist= np.zeros((257,257))
hydroeleg= np.zeros((xglb,yglb))
hydroelegsp= np.zeros((xglb,yglb))

In [38]:
for filename in sorted(file_list, key = last_7chars):
    if filename.endswith(".dat"): # or filename.endswith(".png"):
        globals()["data" + str(i)]= pd.read_csv( filename, sep=' ', header=None, skipinitialspace=True, engine='python')
        a= np.array(globals()["data" + str(i)])
        j=0
        while j <= 7:
            globals()["data" + str(i)+ 'index_' + str(j)] = np.reshape(a[:,j], (257,257))
            #globals()["decidata" + str(i)+ 'index_' + str(j)]= astropy.nddata.block_reduce(globals()["data" + str(i)+ 'index_' + str(j)],block_size=(13,13))
            j+= 1
            
        globals()['hydroele'+ str(i)]=( eval('data' + str(i)+ 'index_0') > eval('data' +str(i)+'index_2'))*1    
        #globals()['lake'+ str(i)]=(globals()["data" + str(i)+ 'index_' + str(1)]>globals()["data" + str(i)+ 'index_' + str(2)])*1
        #p=i h=j
        for p in range(xglb):
            long[p,1:yglb]=(p)*5.0

        for h in range(yglb):
            latg[1:xglb,h]=(h)*5.0-90

        for p in range(xglb):
            for h in range(yglb):
                if latg[p,h] >= 45: #less than -45 for sp
                    dist=np.sqrt((lat-latg[p,h])**2 + (lon-long[p,h])**2)
                    distmin= dist.min()
                    distequal = dist==distmin
                    #latr= lat> latg[p,h] and/& lat<latg[p+1,h] and lon 
                    hydroeleg[p,h]= globals()['hydroele'+ str(i)][distequal]
                    #decilakeg= astropy.nddata.block_reduce(lakeg,block_size=(13,13))

      
        #np.savetxt('//Users//qjf838/Documents/SpaceData/hydroele/NPhydroele'+str(i)+'.txt', hydroeleg, delimiter=',')

        #fig = plt.figure()
        #ax = fig.add_subplot(111)  #cmap=lisbon is nice
        #ax.imshow(lakeg, aspect='auto')#, interpolation='nearest')
        #fig.savefig('//Users//qjf838/Documents/SpaceData/testrun/plot'+str(i)+'.png')

        i += 1
    else:
        continue

In [39]:
NP_hydroele_files = glob2.glob('//Users//qjf838/Documents/SpaceData/Evapnp' + "/*.txt")
hydroeleNP=[]
for txtfile in sorted(NP_evap_files, key = last_8chars):
    if txtfile.endswith(".txt"): # or filename.endswith(".png"):
        df = pd.read_csv(txtfile, index_col=None, header=0)
        hydroeleNP.append(df)
hydroeleNP = pd.concat(hydroeleNP, axis=0, ignore_index=True)

In [40]:
%cd "/Volumes/passport/Present/Work/SpaceLabWork/south_pole/SP_EKM0_TBS1_KP"

/Volumes/passport/Present/Work/SpaceLabWork/south_pole/SP_EKM0_TBS1_KP


In [41]:
isp=1
for filename in sorted(file_list_sp, key = last_7chars):
    if filename.endswith(".dat"): # or filename.endswith(".png"):
        globals()["data" + str(isp)]= pd.read_csv( filename, sep=' ', header=None, skipinitialspace=True, engine='python')
        a= np.array(globals()["data" + str(isp)])
        j=0
        while j <= 7:
            globals()["SPdata" + str(isp)+ 'index_' + str(j)] = np.reshape(a[:,j], (257,257))
            #globals()["decidata" + str(i)+ 'index_' + str(j)]= astropy.nddata.block_reduce(globals()["data" + str(i)+ 'index_' + str(j)],block_size=(13,13))
            j+= 1
            
        globals()['SPhydroele'+ str(isp)]=( eval('SPdata' + str(isp)+ 'index_0') > eval('SPdata' +str(isp)+'index_2'))*1    
        #globals()['lake'+ str(i)]=(globals()["data" + str(i)+ 'index_' + str(1)]>globals()["data" + str(i)+ 'index_' + str(2)])*1
        #p=i h=j
        for p in range(xglb):
            long[p,1:yglb]=(p)*5.0

        for h in range(yglb):
            latg[1:xglb,h]=(h)*5.0-90

        for p in range(xglb):
            for h in range(yglb):
                if latg[p,h] >= 45: #less than -45 for sp
                    dist=np.sqrt((lat-latg[p,h])**2 + (lon-long[p,h])**2)
                    distmin= dist.min()
                    distequal = dist==distmin
                    #latr= lat> latg[p,h] and/& lat<latg[p+1,h] and lon 
                    hydroelegsp[p,h]= globals()['SPhydroele'+ str(isp)][distequal]
                    #decilakeg= astropy.nddata.block_reduce(lakeg,block_size=(13,13))

        
        #np.savetxt('//Users//qjf838/Documents/SpaceData/hydroelesp/SPhydroele'+str(isp)+'.txt', hydroelegsp, delimiter=',')

        #fig = plt.figure()
        #ax = fig.add_subplot(111)  #cmap=lisbon is nice
        #ax.imshow(lakeg, aspect='auto')#, interpolation='nearest')
        #fig.savefig('//Users//qjf838/Documents/SpaceData/testrun/plot'+str(i)+'.png')

        isp += 1
    else:
        continue

In [42]:
SP_hydroele_files = glob2.glob('//Users//qjf838/Documents/SpaceData/Evapsp' + "/*.txt")
hydroeleSP=[]
for txtfile in sorted(SP_evap_files, key = last_8chars):
    if txtfile.endswith(".txt"): # or filename.endswith(".png"):
        df = pd.read_csv(txtfile, index_col=None, header=0)
        hydroeleSP.append(df)
hydroeleSP = pd.concat(hydroeleSP, axis=0, ignore_index=True)

In [43]:
hydroele_combined= hydroeleNP[hydroeleNP== 0]= hydroeleSP[hydroeleNP== False]

In [44]:
%cd "/Volumes/passport/Present/Work/SpaceLabWork/north_pole/NP_EKM0_TBS0_KP"

/Volumes/passport/Present/Work/SpaceLabWork/north_pole/NP_EKM0_TBS0_KP


In [46]:
i = 1
isp=1
xglb= int(360/5)
yglb=int(180/5)
#tglb=84 #I dont think I need it as it will iterate through i
lake_xglb= 0
lake_yglb= 0
lake_tglb= 0
long=np.zeros((xglb,yglb))
latg=np.zeros((xglb,yglb))
dist= np.zeros((257,257))
aquarechargg= np.zeros((xglb,yglb))
aquarechargsp= np.zeros((xglb,yglb))

In [47]:
for filename in sorted(file_list, key = last_7chars):
    if filename.endswith(".dat"): # or filename.endswith(".png"):
        globals()["data" + str(i)]= pd.read_csv( filename, sep=' ', header=None, skipinitialspace=True, engine='python')
        a= np.array(globals()["data" + str(i)])
        j=0
        while j <= 7:
            globals()["data" + str(i)+ 'index_' + str(j)] = np.reshape(a[:,j], (257,257))
            #globals()["decidata" + str(i)+ 'index_' + str(j)]= astropy.nddata.block_reduce(globals()["data" + str(i)+ 'index_' + str(j)],block_size=(13,13))
            j+= 1
            
        globals()['aquare'+ str(i)]=eval('data' + str(i)+ 'index_4')   
        #globals()['lake'+ str(i)]=(globals()["data" + str(i)+ 'index_' + str(1)]>globals()["data" + str(i)+ 'index_' + str(2)])*1
        #p=i h=j
        for p in range(xglb):
            long[p,1:yglb]=(p)*5.0

        for h in range(yglb):
            latg[1:xglb,h]=(h)*5.0-90

        for p in range(xglb):
            for h in range(yglb):
                if latg[p,h] >= 45: #less than -45 for sp
                    dist=np.sqrt((lat-latg[p,h])**2 + (lon-long[p,h])**2)
                    distmin= dist.min()
                    distequal = dist==distmin
                    #latr= lat> latg[p,h] and/& lat<latg[p+1,h] and lon 
                    aquarechargg[p,h]= globals()['aquare'+ str(i)][distequal]
                    #decilakeg= astropy.nddata.block_reduce(lakeg,block_size=(13,13))

        
        #np.savetxt('//Users//qjf838/Documents/SpaceData/aquare/NPaquare'+str(i)+'.txt',aquarechargg, delimiter=',')

        #fig = plt.figure()
        #ax = fig.add_subplot(111)  #cmap=lisbon is nice
        #ax.imshow(lakeg, aspect='auto')#, interpolation='nearest')
        #fig.savefig('//Users//qjf838/Documents/SpaceData/testrun/plot'+str(i)+'.png')

        i += 1
    else:
        continue

In [48]:
NP_aquare_files = glob2.glob('//Users//qjf838/Documents/SpaceData/aquare' + "/*.txt")
aquareNP=[]
for txtfile in sorted(NP_aquare_files, key = last_8chars):
    if txtfile.endswith(".txt"): # or filename.endswith(".png"):
        df = pd.read_csv(txtfile, index_col=None, header=0)
        aquareNP.append(df)
aquareNP = pd.concat(aquareNP, axis=0, ignore_index=True)

In [49]:
%cd "/Volumes/passport/Present/Work/SpaceLabWork/south_pole/SP_EKM0_TBS1_KP"

/Volumes/passport/Present/Work/SpaceLabWork/south_pole/SP_EKM0_TBS1_KP


In [50]:
for filename in sorted(file_list_sp, key = last_7chars):
    if filename.endswith(".dat"): # or filename.endswith(".png"):
        globals()["data" + str(isp)]= pd.read_csv( filename, sep=' ', header=None, skipinitialspace=True, engine='python')
        a= np.array(globals()["data" + str(isp)])
        j=0
        while j <= 7:
            globals()["SPdata" + str(isp)+ 'index_' + str(j)] = np.reshape(a[:,j], (257,257))
            #globals()["decidata" + str(i)+ 'index_' + str(j)]= astropy.nddata.block_reduce(globals()["data" + str(i)+ 'index_' + str(j)],block_size=(13,13))
            j+= 1
            
        globals()['SPaquare'+ str(isp)]= eval('SPdata' + str(isp)+ 'index_0')
        #globals()['lake'+ str(i)]=(globals()["data" + str(i)+ 'index_' + str(1)]>globals()["data" + str(i)+ 'index_' + str(2)])*1
        #p=i h=j
        for p in range(xglb):
            long[p,1:yglb]=(p)*5.0

        for h in range(yglb):
            latg[1:xglb,h]=(h)*5.0-90

        for p in range(xglb):
            for h in range(yglb):
                if latg[p,h] >= 45: #less than -45 for sp
                    dist=np.sqrt((lat-latg[p,h])**2 + (lon-long[p,h])**2)
                    distmin= dist.min()
                    distequal = dist==distmin
                    #latr= lat> latg[p,h] and/& lat<latg[p+1,h] and lon 
                    aquarechargsp[p,h]= globals()['SPaquare'+ str(isp)][distequal]
                    #decilakeg= astropy.nddata.block_reduce(lakeg,block_size=(13,13))

        
        
        #np.savetxt('//Users//qjf838/Documents/SpaceData/aquaresp/SPaquare'+str(isp)+'.txt', aquarechargsp, delimiter=',')

        #fig = plt.figure()
        #ax = fig.add_subplot(111)  #cmap=lisbon is nice
        #ax.imshow(lakeg, aspect='auto')#, interpolation='nearest')
        #fig.savefig('//Users//qjf838/Documents/SpaceData/testrun/plot'+str(i)+'.png')

        isp += 1
    else:
        continue

In [51]:
SP_aquare_files = glob2.glob('//Users//qjf838/Documents/SpaceData/aquaresp' + "/*.txt")
aquareSP=[]
for txtfile in sorted(SP_aquare_files, key = last_8chars):
    if txtfile.endswith(".txt"): # or filename.endswith(".png"):
        df = pd.read_csv(txtfile, index_col=None, header=0)
        aquareSP.append(df)
aquareSP = pd.concat(aquareSP, axis=0, ignore_index=True)

In [52]:
aquare_combined= aquareNP[aquareNP== 0]= aquareSP[aquareNP== False]

In [80]:

allcombined=[combined, evap_combined, hydroele_combined, aquare_combined]
combo= []
for i in allcombined:
    df = pd.DataFrame(i)
    combo.append(df)

In [81]:
combo

[      0.000000000000000000e+00  0.000000000000000000e+00.1  \
 0                          0.0                         0.0   
 1                          0.0                         0.0   
 2                          0.0                         0.0   
 3                          0.0                         0.0   
 4                          0.0                         0.0   
 ...                        ...                         ...   
 5959                       0.0                         0.0   
 5960                       0.0                         0.0   
 5961                       0.0                         0.0   
 5962                       0.0                         0.0   
 5963                       0.0                         0.0   
 
       0.000000000000000000e+00.2  0.000000000000000000e+00.3  \
 0                            0.0                         0.0   
 1                            0.0                         0.0   
 2                            0.0              

In [82]:
np.savetxt('//Users//qjf838/Documents/SpaceData/final?/lake.txt', combined, delimiter=',')
np.savetxt('//Users//qjf838/Documents/SpaceData/final?/evap.txt', evap_combined, delimiter=',')
np.savetxt('//Users//qjf838/Documents/SpaceData/final?/hydroele.txt',hydroele_combined, delimiter=',')
np.savetxt('//Users//qjf838/Documents/SpaceData/final?/aquare.txt', aquare_combined, delimiter=',')